In [1]:
from generatefeatures import generate_features
import pandas as pd
import numpy as np
import os
import rebound
from collections import OrderedDict

First we generate all the features

In [2]:
dflag = 1

if dflag == 0:
    d = rebound.Simulation.from_file('ic0000000.bin')
else:
    columns = ["m1","T1","P1","h1","k1","m2","T2","P2","h2","k2","m3","T3","P3","h3","k3"] #h,k=esinw,ecosw
    d = pd.read_csv("systems/KOI-2086.dat",names=columns,sep="\s+").iloc[0]

In [3]:
%%time
fullfeatures = generate_features(d)

/Users/silburt/phd/rebound/rebound/simulation.py:508: RuntimeWarning: At least 10 predictor corrector loops in IAS15 did not converge. This is typically an indication of the timestep being too large.
  warnings.warn(msg[1:], RuntimeWarning)


CPU times: user 2.99 s, sys: 36.9 ms, total: 3.03 s
Wall time: 3.08 s


Load the trained model

In [5]:
import pickle 
model = pickle.load(open('OptimalXGBmodel.pkl', 'rb'))

/Users/silburt/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/silburt/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.18. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


The trained model only uses a subset of the generated features, so make a list of the ones it needs for evaluation

In [6]:
import pandas as pd
features = []
features += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['Lyapunov_time']

There's probably a better way to do this, but it seems the XGBoost API wants a dataframe passed to its predict method. So we need to turn our features into a dataframe with a single row. 

In [8]:
test = pd.DataFrame(columns=features)
test.loc[0] = fullfeatures[features] # take only the features the model uses
test

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,max_ecross2,min_ecross2,avg_ecross3,std_ecross3,max_ecross3,min_ecross3,norm_a1_slope,norm_a2_slope,norm_a3_slope,Lyapunov_time
0,2.179294,0.0,7.02901,49.203071,2.000826,4.001652,7.233072,50.631505,2.006466,4.012932,...,280810.365604,0.346762,0.0,0.0,72.156392,0.258876,-0.000006,-0.000006,1.747732e-07,249955.313923


Predict probability

In [24]:
model.predict_proba(test)

array([[  9.99889970e-01,   1.10002678e-04]], dtype=float32)